<a href="https://colab.research.google.com/github/Zrce/Google-Discover-API-in-Python/blob/main/Google_Discover_API_in_Python_pushing_to_Pandas_Data_Frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import os
from google.colab import drive
import json
import pandas as pd
import datetime as DT

In [18]:
#Start and end date
today = DT.date.today()
week_ago_start = today - DT.timedelta(days=9)
week_ago_end = today - DT.timedelta(days=2)
print(week_ago_start)
print(week_ago_end)

2021-10-17
2021-10-24


In [19]:
#Access Drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/GSC/")
os.listdir()
#Get GSC key file in the same folder
key = 'gscblick-590cb1886f0d.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
#Connect GSC 
def connect(key):
  scope = ['https://www.googleapis.com/auth/webmasters']
  credentials = service_account.Credentials.from_service_account_file(key, scopes=scope)
  service = build('searchconsole','v1',credentials=credentials)
  return service

In [21]:
def query(service, site_url, payload):
  response = service.searchanalytics().query(siteUrl=site_url, body=payload).execute()
  print(response)
  results = []

  for row in response['rows']:    
    data = {}
    
    for i in range(len(payload['dimensions'])):
      data[payload['dimensions'][i]] = row['keys'][i]

    data['clicks'] = row['clicks']
    data['impressions'] = row['impressions']
    results.append(data)
  
  return pd.DataFrame.from_dict(results)

In [26]:
def queryDiscover():
  service = connect(key)
  payload = {
    'startDate': str(week_ago_start),
    'endDate': str(week_ago_end),
    'dimensions': ["page"],
    'type': 'discover'
  }

  site_url = "https://www.blick.ch/"
  df = query(service, site_url, payload)
  return df

In [ ]:
dfDiscover = queryDiscover()
dfDiscover